In [41]:
import os

import numpy as np
import torch
#import jiwer : label 텍스트가 있을 경우 활용
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from typing import Any, Text

class sttmodel:
    def __init__(self):
        self.model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
        self.processor = WhisperProcessor.from_pretrained("openai/whisper-small")
        self.device = None
        self.transcription = None
        self.sr = 16000

    def inference_stt(self, data):
        audio_array = data
        input_features = self.processor.feature_extractor(
            audio_array,
            sampling_rate = self.sr,
            return_tensors = "pt"
        ).input_features
        # 입력 특성을 모델과 동일한 장치로 이동
        input_features = input_features.to(self.device)

        # 모델을 통한 예측
        with torch.no_grad():
            predicted_ids = self.model.generate(input_features)

        # 예측된 토큰을 텍스트로 디코딩
        transcription = self.processor.tokenizer.batch_decode(
            predicted_ids,
            skip_special_tokens=True
        )[0]

        return transcription

    def check_cuda(self,):
        if torch.cuda.is_available():
            self.device = torch.device("cuda:0")
            print("GPU 사용")
        else:
            self.device = torch.device("cpu")
            print("CPU 사용")

    def load_pt(self, pt_file: str):
        try:
            print(f"모델 파일 로드 시도: {pt_file}")
            state_dict = torch.load(pt_file, map_location=self.device)
            self.model.load_state_dict(state_dict)
            print(f"모델 파일 성공적으로 로드됨: {pt_file}")
            return True
        except Exception as e:
            print(f"모델 파일 에러: {str(e)}")
            return False

    def find_pt(self, model_dir):
        found = False
        if os.path.exists(model_dir):
            print(f"{model_dir} 디렉토리 존재함")
            # 먼저 디렉토리 내용 출력
            print(f"디렉토리 내용: {os.listdir(model_dir)}")

            for root, _, files in os.walk(model_dir):
                for file in files:
                    if file.endswith(".pt"):
                        full_path = os.path.join(root, file)
                        print(f"모델 파일 발견: {full_path}")
                        if self.load_pt(full_path):
                            found = True
                            break
                if found:
                    break
        else:
            print(f"{model_dir} 디렉토리가 존재하지 않습니다.")

        return found

    def stt_model(self, test_file):
        self.model.eval()
        self.transcription = self.inference_stt(test_file)

    def start_stt(self, test_file: np.ndarray) -> Any:
        self.check_cuda()
        self.find_pt("/content/drive/MyDrive/model")   # pt 확장자로 된 pt파일 이름을 넣는 것이 아닌 pt 파일이 존재하는 디렉토리 이름을 넣는다.
        self.stt_model(test_file)
        return self.transcription


In [4]:
stt = sttmodel()

In [57]:
import os
import numpy as np
import torch.nn as nn
import torch
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from typing import Any, Text

class sttmodel(nn.Module):
    def __init__(self):
        super(sttmodel, self).__init__()
        # 기본 모델 구조만 초기화하고, 가중치는 나중에 로드
        self.model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
        self.processor = WhisperProcessor.from_pretrained("openai/whisper-small")
        self.device = None
        self.transcription = None
        self.sr = 16000

    def inference_stt(self, data):
        audio_array = data
        input_features = self.processor.feature_extractor(
            audio_array,
            sampling_rate = self.sr,
            return_tensors = "pt"
        ).input_features
        input_features = input_features.to(self.device)

        forced_decoder_ids = self.processor.get_decoder_prompt_ids(language="korean", task="transcribe")

        with torch.no_grad():
            predicted_ids = self.model.generate(
                input_features,
                forced_decoder_ids=forced_decoder_ids,
                language="ko",  # 한국어 지정
                task="transcribe"
            )
        transcription = self.processor.tokenizer.batch_decode(
            predicted_ids,
            skip_special_tokens=True
        )[0]

        return transcription

    def check_cuda(self,):
        if torch.cuda.is_available():
            self.device = torch.device("cuda:0")
            print("GPU 사용")
        else:
            self.device = torch.device("cpu")
            print("CPU 사용")

    def load_pt(self, pt_file: str):
        try:
            print(f"모델 파일 로드 시도: {pt_file}")
            state_dict = torch.load(pt_file, map_location=self.device)
            self.model.load_state_dict(state_dict)
            print(f"모델 파일 성공적으로 로드됨: {pt_file}")
            return True
        except Exception as e:
            print(f"모델 파일 에러: {str(e)}")
            return False

    def find_pt(self, model_dir):
        found = False
        if os.path.exists(model_dir):
            print(f"{model_dir} 디렉토리 존재함")
            # 먼저 디렉토리 내용 출력
            print(f"디렉토리 내용: {os.listdir(model_dir)}")

            for root, _, files in os.walk(model_dir):
                for file in files:
                    if file.endswith(".pt"):
                        full_path = os.path.join(root, file)
                        print(f"모델 파일 발견: {full_path}")
                        if self.load_pt(full_path):
                            found = True
                            break
                if found:
                    break
        else:
            print(f"{model_dir} 디렉토리가 존재하지 않습니다.")

        return found

    def stt_model(self, test_file):
        self.model.eval()
        self.transcription = self.inference_stt(test_file)

    def start_stt(self, test_file: np.ndarray) -> Any:
        self.check_cuda()
        # 절대 경로로 명확히 지정
        model_path = "/content/drive/MyDrive/model"
        # 모델 로드 시도
        model_loaded = self.find_pt(model_path)

        if not model_loaded:
            raise ValueError("학습된 모델 파일을 찾거나 로드할 수 없습니다.")

        self.stt_model(test_file)
        return self.transcription

In [20]:
cd /content/drive/MyDrive

/content/drive/MyDrive


In [61]:
import os
file_path = '/content/drive/MyDrive/KsponSpeech_data/10.한국어음성/KsponSpeech_data_unzip_data/KsponSpeech_01/KsponSpeech_0001/KsponSpeech_000002.pcm'

with open(file_path, 'rb') as f:
    pcm_data = f.read()

In [62]:
import numpy as np

# PCM 파일을 NumPy 배열로 변환
pcm_data_np = np.frombuffer(pcm_data, dtype=np.int16).astype(np.float32) / 32768.0
stt = sttmodel().to('cuda')
stt.start_stt(pcm_data_np)

GPU 사용
/content/drive/MyDrive/model 디렉토리 존재함
디렉토리 내용: ['trainedModel.pt', 'example.pcm', '.ipynb_checkpoints']
모델 파일 발견: /content/drive/MyDrive/model/trainedModel.pt
모델 파일 로드 시도: /content/drive/MyDrive/model/trainedModel.pt
모델 파일 성공적으로 로드됨: /content/drive/MyDrive/model/trainedModel.pt


'나는 악습은 원래 없어진다, 없어져야 된다고 생각하긴 했는데 근데 그에 약간 필요악으로 하나 정도쯤은 있어야 되거든. 물 뜨러 가고. bd'

In [34]:
import os
# 현재 작업 디렉토리 확인
print("현재 디렉토리:", os.getcwd())
# 'model' 디렉토리가 있는지 확인
print("model 디렉토리 존재 여부:", os.path.exists("model"))
# 'model' 디렉토리에 무슨 파일이 있는지 확인
if os.path.exists("model"):
    print("model 디렉토리 내용:", os.listdir("model"))

현재 디렉토리: /content/drive/MyDrive
model 디렉토리 존재 여부: True
model 디렉토리 내용: ['trainedModel.pt', 'example.pcm', '.ipynb_checkpoints']
